In [1]:
import pandas as pd
data = pd.read_csv(r'C:\Users\sigal\OneDrive - post.bgu.ac.il\sigal\submissions\journal\data\taxifull\train.csv',nrows=300000)
data.shape

(300000, 9)

In [2]:
data.head(100)

,TRIP_ID,CALL_TYPE,ORIGIN_CALL,ORIGIN_STAND,TAXI_ID,TIMESTAMP,DAY_TYPE,MISSING_DATA,POLYLINE
0,1372636858620000589,C,NaN,NaN,20000589,1372636858,A,False,"[[-8.618643,41.141412],[-8.618499,41.141376],[..."
1,1372637303620000596,B,NaN,7.0,20000596,1372637303,A,False,"[[-8.639847,41.159826],[-8.640351,41.159871],[..."
2,1372636951620000320,C,NaN,NaN,20000320,1372636951,A,False,"[[-8.612964,41.140359],[-8.613378,41.14035],[-..."
3,1372636854620000520,C,NaN,NaN,20000520,1372636854,A,False,"[[-8.574678,41.151951],[-8.574705,41.151942],[..."
4,1372637091620000337,C,NaN,NaN,20000337,1372637091,A,False,"[[-8.645994,41.18049],[-8.645949,41.180517],[-..."
5,1372636965620000231,C,NaN,NaN,20000231,1372636965,A,False,"[[-8.615502,41.140674],[-8.614854,41.140926],[..."
6,1372637210620000456,C,NaN,NaN,20000456,1372637210,A,False,"[[-8.57952,41.145948],[-8.580942,41.145039],[-..."
7,1372637299620000011,C,NaN,NaN,20000011,1372637299,A,False,"[[-8.617563,41.146182],[-8.617527,41.145849],[..."
8,1372637274620000403,C,NaN,NaN,20000403,1372637274,A,False,"[[-8.611794,41.140557],[-8.611785,41.140575],[..."
9,1372637905620000320,C,NaN,NaN,20000320,1372637905,A,False,"[[-8.615907,41.140557],[-8.614449,41.141088],[..."


Filter only valid records

In [9]:
data = data[~data['MISSING_DATA'] ] #remove records with missing data
data = data[data['POLYLINE'].str.len()>8]
data = data[((data['POLYLINE'].str.find('-8.5')>0) | (data['POLYLINE'].str.find('-8.6')>0) )]
data = data[((data['POLYLINE'].str.find('41.1')>0) )]
#data = data.reindex()

In [10]:
data.shape

(297964, 9)

Converting the data to record per point

In [11]:
import datetime
from datetime import timedelta
import numpy as np
def splitPolyline(poly,d,trip,taxi,dayType):
    x = [r.split(',') for r in poly.replace('[[','').replace(']]','').split('],[')]
    x= np.asmatrix(x) #convert polyline to matrix of points
    x=pd.DataFrame(x.astype('float').round(2)) #discretize to 2 digits after the dot
    x.columns = ['lon','lat']
    x['timestamp'] =  timedelta(seconds=15) * x.index.values + datetime.datetime.fromtimestamp(d)#calculate timestamp
    x = x[0:len(x)-1:4].append(x.loc[len(x)-1]) # sample every fourth record, and the destination record
    x['TRIP_ID'] = trip
    x['TAXI_ID'] = taxi
    x['DAY_TYPE'] = dayType
    return x;


In [12]:
data1=[]
for i in range(len(data)):#loading 300000 first trips
    rec = data.iloc[i]
    newData = splitPolyline(rec['POLYLINE'],rec['TIMESTAMP'],rec['TRIP_ID'],rec['TAXI_ID'],rec['DAY_TYPE'])
    # Filter a 20km*20km grid (-8.6,41.1),(-8.4,41.3)
    #newData = newData[(newData['lon']>=-8.6) & (newData['lon']<=-8.4) & (newData['lat']>=41.1) & (newData['lat']<=41.3)]
    if newData is not None:
        if (i==1):
            data1 = newData
        else:
            data1=data1.append(newData)  
data1.shape


(3839680, 6)

number of states for the loaded data:

In [ ]:
len(set(data1['lon'].astype('str')+'_'+data1['lat'].astype('str')))

filtering a 20km*20km grid

In [20]:
data2 = data1[(data1['lon']>-8.65) & (data1['lon']<=-8.57) & (data1['lat']>=41.13) & (data1['lat']<=41.18)]
data2.shape

(2790775, 6)

This is the number of states in the filtered area

In [19]:
len(set(data2['lon'].astype('str')+'_'+data2['lat'].astype('str')))

40

In [27]:
data2.to_csv(r'C:\Users\sigal\OneDrive - post.bgu.ac.il\sigal\submissions\journal\data\Taxi_300000_FixedGrid.txt.gz',index=False,compression='gzip')